ERROR: You must give at least one requirement to install (see "pip help install")


In [27]:
import os
import numpy as np
import torch
import pandas as pd
from functools import reduce
from transformers import AutoTokenizer

In [28]:
device = torch.device("cpu")

In [45]:
d_path = os.path.join(os.getcwd(), "..", "..", "token-classes-dataset")

In [46]:
# list of partial dataframes
dfs = []

# collects partial dataframes from oracles
for file_name in os.listdir(d_path):
    df = pd.read_json(os.path.join(d_path,  file_name))
    dfs.append(df)

df_dataset = pd.concat(dfs)
# drop column id (it is not relevant for training the model)
df_dataset = df_dataset.drop(['id'], axis=1)
# map empty cells to empty strings
df_dataset.fillna('', inplace=True)
# specify the type of each column in the dataset
df_dataset = df_dataset.astype({
    'label': 'bool',
    'oracleId': 'int64',
    'oracleType': 'string',
    'projectName': 'string',
    'packageName': 'string',
    'className': 'string',
    'javadocTag': 'string',
    'methodJavadoc': 'string',
    'methodSourceCode': 'string',
    'classJavadoc': 'string',
    'classSourceCode': 'string',
    'oracleSoFar': 'string',
    'token': 'string',
    'tokenClass': 'string',
    'tokenInfo': 'string'
})

In [48]:
df_dataset.shape

(88262, 15)

In [43]:
unique_values = df_dataset['tokenClass'].unique()

In [44]:
unique_values

<StringArray>
[        'MethodArgument',      'NonEqIneqOperator',                  'S_INT',
              'Semicolon',             'EqOperator',                 'DOUBLE',
                   'NULL',        'LogicalOperator',           'QuestionMark',
         'MethodResultID',                  'Colon',                   'TRUE',
     'OpeningParenthesis',     'ClosingParenthesis',                  'FALSE',
                   'This',                 'Period',             'MethodName',
                'BOOLEAN',             'ClassField',                  'Class',
           'IneqOperator',            'ArraysClass',           'StreamMethod',
            'MatchMethod',         'MatchMethodVar',             'RightArrow',
   'ArithmeticalOperator',                  'Comma',   'BitwiseShiftOperator',
 'BitwiseLogicalOperator',               'S_STRING',     'InstanceOfOperator']
Length: 33, dtype: string

In [22]:
df_dataset.columns

Index(['label', 'oracleId', 'oracleType', 'projectName', 'packageName',
       'className', 'javadocTag', 'methodJavadoc', 'methodSourceCode',
       'classJavadoc', 'classSourceCode', 'oracleSoFar', 'token', 'tokenClass',
       'tokenInfo'],
      dtype='object')

In [23]:
# delete the oracle ids and the tgt labels from the input dataset
df_src = df_dataset.drop(['label','oracleId','projectName','classSourceCode','classJavadoc','token','tokenInfo'], axis=1)

In [24]:
df_src.shape

(88262, 8)

In [25]:
%%capture
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base", model_max_length=512)

In [26]:
df_src_concat = df_src.apply(lambda row: tokenizer.tokenize(tokenizer.cls_token.join(row.values)), axis=1)
# The pandas dataframe is transformed in a list of strings: each string is a input
# to the model
src = df_src_concat.to_numpy().tolist()

#max_len = reduce(lambda max_len, s: len(s) if len(s) > max_len else max_len, src,0) + 2

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


In [11]:
unique_values = df_dataset['tokenClass'].unique()

In [12]:
unique_values

<StringArray>
[        'MethodArgument',                  'Class',              'Semicolon',
     'OpeningParenthesis',                   'TRUE',            'ArraysClass',
                   'This',           'QuestionMark',         'MethodResultID',
           'IneqOperator',             'EqOperator',                 'Period',
     'InstanceOfOperator',             'MethodName',                   'NULL',
     'ClosingParenthesis',                  'Colon',        'LogicalOperator',
                  'FALSE',      'NonEqIneqOperator',                  'S_INT',
  'BitwiseNegateOperator',                 'DOUBLE',   'ArithmeticalOperator',
   'BitwiseShiftOperator',               'S_STRING',                'BOOLEAN',
           'StreamMethod',            'MatchMethod',         'MatchMethodVar',
             'RightArrow',             'ClassField',          'ClassModifier',
 'BitwiseLogicalOperator',                  'Comma']
Length: 35, dtype: string

In [13]:
max_len

NameError: name 'max_len' is not defined

In [ ]:
df_src.head()

,oracleType,packageName,className,javadocTag,oracleSoFar,tokenClass
0,PRE,plume,Lookup,@param reader where to read the entry from,,MethodArgument
1,PRE,plume,Lookup,@param reader where to read the entry from,,Class
2,PRE,plume,Lookup,@param reader where to read the entry from,,Semicolon
3,PRE,plume,Lookup,@param reader where to read the entry from,,OpeningParenthesis
4,NORMAL_POST,plume,Lookup,"the next entry, or null",,MethodArgument


In [ ]:
max(src)

'PRE<s>plume<s>WeakIdentityPair<s>@param b second argument<s><s>Semicolon'

In [ ]:
# Assuming you have a list of strings named 'string_list'
max_length_element = max(src, key=lambda x: len(x))

# If you want to get the index of the element instead, you can use argmax()
max_length_index = np.argmax([len(x) for x in src])

In [ ]:
max_length_element

'PRE<s>org.apache.commons.math3.ml.neuralnet.twod.util<s>UnifiedDistanceMatrix<s>@param individualDistances If {@code true}, the 8 individual\ninter-units distances will be {@link #computeImage(NeuronSquareMesh2D)\ncomputed}.  They will be stored in additional pixels around each of\nthe original units of the 2D-map.  The additional pixels that lie\nalong a "diagonal" are shared by <em>two</em> pairs of units: their\nvalue will be set to the average distance between the units belonging\nto each of the pairs.  The value zero will be stored in the pixel\ncorresponding to the location of a unit of the 2D-map.\n<br>\nIf {@code false}, only the average distance between a unit and all its\nneighbours will be computed (and stored in the pixel corresponding to\nthat unit of the 2D-map).  In that case, the number of neighbours taken\ninto account depends on the network\'s\n{@link org.apache.commons.math3.ml.neuralnet.SquareNeighbourhood\nneighbourhood type}.<s><s>OpeningParenthesis'

In [ ]:
len(max_length_element)

970

In [ ]:
b = []

b.append(max_length_element)

a = tokenizer(
    b,
    truncation=False
)

In [ ]:
len(a["input_ids"][0])

268

In [ ]:
# Assuming you have a string named 'my_string'
numerical_representation = [ord(char) for char in my_string]  # Convert string to a list of ASCII values

# Create a PyTorch tensor from the numerical representation
tensor = torch.tensor(numerical_representation)

TypeError: new(): invalid data type 'str'

In [ ]:
type(tokenizer.tokenize(max_length_element)[0])

str

In [ ]:
max_len

268

In [ ]:
string_lengths = np.array([len(sublist) + 2 for sublist in src ])

max_length = np.max(string_lengths)
mean_length = np.mean(string_lengths)
median_length = np.median(string_lengths)

In [ ]:
max_length

27727

In [ ]:
mean_length

215.18788288835913

In [ ]:
median_length

118.0

In [28]:
filtered_strings = [string for string in src if len(string) < 512]

In [ ]:
len(src)

189187

In [ ]:
len(filtered_strings)

155578

In [ ]:
len(df_src["tokenClass"].value_counts())

35

In [19]:
df_dataset["tokenClass"].value_counts()#/len(df_dataset["tokenClass"]) * 100

Class                     32428
Semicolon                 31038
MethodArgument            29226
OpeningParenthesis        29003
This                      14980
TRUE                      12001
ArraysClass                6109
LogicalOperator            5729
EqOperator                 5149
IneqOperator               4098
Period                     3778
InstanceOfOperator         2651
NULL                       2358
S_INT                      2212
ClosingParenthesis         1368
NonEqIneqOperator          1184
MethodResultID              961
DOUBLE                      887
BitwiseNegateOperator       841
FALSE                       828
MethodName                  523
Colon                       386
QuestionMark                296
ArithmeticalOperator        253
ClassField                  239
BitwiseLogicalOperator      186
BitwiseShiftOperator        186
BOOLEAN                     138
StreamMethod                 52
MatchMethodVar               42
MatchMethod                  14
RightArr

In [17]:
np.bincount(df_src["tokenClass"])

ValueError: invalid literal for int() with base 10: 'MethodArgument'

In [14]:
# Get unique values from the column
unique_values = df_dataset['tokenClass'].unique()

# Create a dictionary to map string values to their corresponding vector
mapping = {}
for i, value in enumerate(unique_values):
    vector = np.zeros(len(unique_values))
    vector[i] = 1.0
    mapping[value] = list(vector)

# Add a new column to the original DataFrame with the mapped vectors
df_dataset['tokenClassVectorized'] = df_dataset['tokenClass'].map(mapping)

In [15]:
df_dataset.head()

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo,tokenClassVectorized
0,False,15264,PRE,plume-lib-1.1.0,plume,Lookup,@param reader where to read the entry from,/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,MethodArgument,,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,False,15264,PRE,plume-lib-1.1.0,plume,Lookup,@param reader where to read the entry from,/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,Class,,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,True,15264,PRE,plume-lib-1.1.0,plume,Lookup,@param reader where to read the entry from,/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,Semicolon,,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,False,15264,PRE,plume-lib-1.1.0,plume,Lookup,@param reader where to read the entry from,/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,OpeningParenthesis,,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,False,15265,NORMAL_POST,plume-lib-1.1.0,plume,Lookup,"the next entry, or null",/** * Returns the next entry. If no more ...,public static Entry old_get_entry(EntryReader ...,"/** * Lookup searches a set of files, much li...",package plume; import static plume.EntryReade...,,,MethodArgument,,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [18]:
# Convert string labels to integer labels
unique_labels, integer_labels = np.unique(df_dataset["tokenClass"], return_inverse=True)

# Count the occurrences of each class
class_counts = np.bincount(integer_labels)

# Calculate the inverse class frequencies
total_samples = np.sum(class_counts)
class_frequencies = class_counts / total_samples
class_weights = 1.0 / class_frequencies

# Normalize the class weights
class_weights /= np.sum(class_weights)

class_weights.sum()

1.0000000000000002

In [25]:
np.unique(df_dataset["tokenClass"], return_counts=True)

(array(['ArithmeticalOperator', 'ArraysClass', 'BOOLEAN',
        'BitwiseLogicalOperator', 'BitwiseNegateOperator',
        'BitwiseShiftOperator', 'Class', 'ClassField', 'ClassModifier',
        'ClosingParenthesis', 'Colon', 'Comma', 'DOUBLE', 'EqOperator',
        'FALSE', 'IneqOperator', 'InstanceOfOperator', 'LogicalOperator',
        'MatchMethod', 'MatchMethodVar', 'MethodArgument', 'MethodName',
        'MethodResultID', 'NULL', 'NonEqIneqOperator',
        'OpeningParenthesis', 'Period', 'QuestionMark', 'RightArrow',
        'S_INT', 'S_STRING', 'Semicolon', 'StreamMethod', 'TRUE', 'This'],
       dtype=object),
 array([  253,  6109,   138,   186,   841,   186, 32428,   239,     7,
         1368,   386,     9,   887,  5149,   828,  4098,  2651,  5729,
           14,    42, 29226,   523,   961,  2358,  1184, 29003,  3778,
          296,    14,  2212,    13, 31038,    52, 12001, 14980]))

In [22]:
len(unique_labels)

35

In [11]:
a = df_dataset[df_dataset["tokenClass"]=="Class"]

In [14]:
a

,label,oracleId,oracleType,projectName,packageName,className,javadocTag,methodJavadoc,methodSourceCode,classJavadoc,classSourceCode,oracleSoFar,token,tokenClass,tokenInfo
143,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,RealFieldElement,Class,"['org.apache.commons.math3', 'RealFieldElement']"
144,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,Primes,Class,"['org.apache.commons.math3.primes', 'Primes']"
145,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,SmallPrimes,Class,"['org.apache.commons.math3.primes', 'SmallPrim..."
146,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,PollardRho,Class,"['org.apache.commons.math3.primes', 'PollardRho']"
147,False,813,EXCEPT_POST,commons-math3-3.6.1,org.apache.commons.math3.linear,MatrixUtils,@throws org.apache.commons.math3.linear.NonSym...,/** * Checks whether a matrix is symmetr...,public static void checkSymmetric(RealMatrix m...,/** * A collection of static methods that ope...,/* * Licensed to the Apache Software Foundati...,(,CurveFitter,Class,"['org.apache.commons.math3.fitting', 'CurveFit..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,False,22109,NORMAL_POST,commons-collections4-4.1,org.apache.commons.collections4,BagUtils,an empty sorted Bag.,/** * Get an empty <code>SortedBag</code...,public static <E> SortedBag<E> emptySortedBag(...,/** * Provides utility methods and decorators...,/* * Licensed to the Apache Software Foundati...,true?methodResultID.equals(,AbstractUntypedIteratorDecorator,Class,"['org.apache.commons.collections4.iterators', ..."
1463,False,22109,NORMAL_POST,commons-collections4-4.1,org.apache.commons.collections4,BagUtils,an empty sorted Bag.,/** * Get an empty <code>SortedBag</code...,public static <E> SortedBag<E> emptySortedBag(...,/** * Provides utility methods and decorators...,/* * Licensed to the Apache Software Foundati...,true?methodResultID.equals(,AbstractOrderedMapIteratorDecorator,Class,"['org.apache.commons.collections4.iterators', ..."
1464,False,22109,NORMAL_POST,commons-collections4-4.1,org.apache.commons.collections4,BagUtils,an empty sorted Bag.,/** * Get an empty <code>SortedBag</code...,public static <E> SortedBag<E> emptySortedBag(...,/** * Provides utility methods and decorators...,/* * Licensed to the Apache Software Foundati...,true?methodResultID.equals(,FilterListIterator,Class,"['org.apache.commons.collections4.iterators', ..."
1465,False,22109,NORMAL_POST,commons-collections4-4.1,org.apache.commons.collections4,BagUtils,an empty sorted Bag.,/** * Get an empty <code>SortedBag</code...,public static <E> SortedBag<E> emptySortedBag(...,/** * Provides utility methods and decorators...,/* * Licensed to the Apache Software Foundati...,true?methodResultID.equals(,SplitMapUtils,Class,"['org.apache.commons.collections4', 'SplitMapU..."
